ผมจะยกตัวอย่างการทำงานของทั้ง 5 ขั้นตอนดังนี้
1. Create a Question (ตั้งโจทย์ว่าเราอยากทำอะไร ต้องหาข้อมูลอะไร ต้องวิเคราะห์รูปแบบไหน)
2. Extract Data (หาข้อมูล)
3. Data Preparation + Data Cleansing (จัดเตรียมข้อมูลในมีความเหมาะสม)
4. Data Analyst (วิเคราะห์ข้อมูล)
5. Presentation (นำเสนอข้อมูล)
---

# 1. Create a Question

โจทย์ :  Youtube ช่อง gssspotted คอมเม้นส่วนใหญ่มาแนวโน้มจะ toxic หรือไม่  <br>
ในตัวอย่างจะขอทำแค่คลิป "สิ่งมหัศจรรย์สิ่งที่ 8 ของโลก (บ้านเอียง) | นิทานก่อนนอนกับลุงไนท์" -> https://www.youtube.com/watch?v=pxFYvpSIVXI 


จากโจทย์ทำให้ผมคิดเป็นขั้นตอนดังนี้
1. แหล่งข้อมูลที่ต้องหาคือ Youtube ช่อง gssspotted
2. อยากรู้ว่าคอมเม้น toxic หรือไม่ ต้องมีการเก็บข้อมูลเกี่ยวกับ"การแสดงความเห็น" เป็นหลัก
3. จะได้ข้อมูลมามี Web scraping ด้วยตนเอง, API (ใช้วิธีนี้ในรอบนี้),ใช้ tools 
3. นอกจากนั้นเราต้องมีการ Label เพิ่มเติมว่าข้อมูลดังกล่าวเป็นข้อความประเภทใด เช่น positive, negative, neutral
4. จะ label ข้อ 3 มีวิธีสองวิธีคือทำเอง และให้ Generative AI (Bard, ChatGPT, etc.)
5. หลังจากได้ข้อมูลเราควร Clean data ด้วยนะ ซึ่งในที่นี้คงทำเพียงคัดแยกความเห็นที่ดูมีท่าทางเป็นสแปม
6. ลอง plot ข้อมูลคร่าวๆก่อนไปทำ Dashboard 
7. ไปทำ Dashboard ด้วย Looker Studio ละกัน เพราะว่า `FREE`
---

# 2. Extract Data

$ pip install --upgrade google-api-python-client

$ pip install --upgrade google-auth-oauthlib google-auth-httplib2

In [34]:
# Documentation 
# https://developers.google.com/youtube/v3/getting-started
# https://developers.google.com/youtube/v3/docs/commentThreads/list 

import os
import googleapiclient.discovery

os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = "YOUR_API_KEY" # ทำตามคลิปนี้ได้เลย https://www.youtube.com/watch?v=XTjtPc0uiG8

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey = DEVELOPER_KEY)

request = youtube.commentThreads().list(
    part = 'id,replies,snippet' ,
    videoId = 'pxFYvpSIVXI', # จากโจทย์ https://www.youtube.com/watch?v=pxFYvpSIVXI ดูไอดีได้ตรง v={videoId}
    maxResults = 100 # ได้มากสุดแค่ 100 comments ถ้าอยากได้มากกว่านี้ต้องเขียนเพิ่มเช่น https://stackoverflow.com/questions/65762711/youtube-data-api-how-to-extract-more-than-100-comments 
)
response = request.execute()

In [37]:
response

{'kind': 'youtube#commentThreadListResponse',
 'etag': '_BwwhqXKwvf09BsBzH9R84yM458',
 'nextPageToken': 'QURTSl9pMnZTNnNQbEZSeDZ3S1pkT3NGYTJZVGxDQWdLbWVLSEhsWHlfYk41TndpN1ByZzRPY2Z4aUVkWnZrRkd6M2ZTWkxpUkYxTnFwZw==',
 'pageInfo': {'totalResults': 100, 'resultsPerPage': 100},
 'items': [{'kind': 'youtube#commentThread',
   'etag': 'MNMqBAJuDc8GIgaS1x0JwbToZt8',
   'id': 'UgxgIbfcSZMyAzc5dxp4AaABAg',
   'snippet': {'channelId': 'UC8uxFKMADMhYn80nkfpPa-w',
    'videoId': 'pxFYvpSIVXI',
    'topLevelComment': {'kind': 'youtube#comment',
     'etag': 'FB77ooBGCZqumSFSDcFWMWWFhCw',
     'id': 'UgxgIbfcSZMyAzc5dxp4AaABAg',
     'snippet': {'channelId': 'UC8uxFKMADMhYn80nkfpPa-w',
      'videoId': 'pxFYvpSIVXI',
      'textDisplay': 'ชอบฟังมาก เพลินสุด ชอบตั้งแต่เล่น raknalok',
      'textOriginal': 'ชอบฟังมาก เพลินสุด ชอบตั้งแต่เล่น raknalok',
      'authorDisplayName': 'Woraporn Keawma',
      'authorProfileImageUrl': 'https://yt3.ggpht.com/PF1mX6KxEN9YOiJKNdr56qsQKMm2EOUGu9CXO3K2x98SY3VChjfj

In [38]:
# จะเห็นว่ามี 100 คอมเม้นตามตัวแปร maxResults ที่ตั้งไว้
len(response['items'])

100

In [68]:
import pandas as pd

name = []
text = []
for i in response['items']:
    authorDisplayName = i['snippet']['topLevelComment']['snippet']['authorDisplayName']
    textDisplay = i['snippet']['topLevelComment']['snippet']['textDisplay']
    name.append(authorDisplayName)
    text.append(textDisplay)

df =pd.DataFrame({'Name':name, 'Comments':text})
df.head(10)

,Name,Comments
0,Woraporn Keawma,ชอบฟังมาก เพลินสุด ชอบตั้งแต่เล่น raknalok
1,Phanuwat Benkasem,พวกคุณมันแค่เด็กในเมือง 555 ไอเหี้ยวันนั้นถ้าอ...
2,Mxx K,เสื้อลุงไนท์หาซื้อจากไหนได้บ้างครับ555
3,หยุดเหมียวไม่ได้,"<a href=""https://www.youtube.com/watch?v=pxFYv..."
4,ฬิน,ขำเกิน ปวดท้อง
5,ูLightvan007,"<a href=""https://www.youtube.com/watch?v=pxFYv..."
6,Razen,บ้านพี่ขยะหน่ะมัน คาด เดา ไม่ ได้~~~~
7,SKT_X,กายนี่คือกายไหนครับเจ้าของบ้าน
8,Sosoมัง,"<a href=""https://www.youtube.com/watch?v=pxFYv..."
9,Fifa Thanakrit,คิดในแง่ดีบ้านเอียงก็ทำให้ได้คอนเทนไปตลอดในวัน...


In [95]:
# คอมเม้นเป็นลิ้งคือเขาแปะช่วงเวลานั้นๆที่เขาสนใจในคลิปกันเฉยๆหนะ อย่างเช่นอันนี้ก็คือลิ้งไประยะเวลาที่ 1.42.29 ของคลิป
print(df[df['Name']=='ูLightvan007']['Comments'].values[0])

<a href="https://www.youtube.com/watch?v=pxFYvpSIVXI&amp;t=1h42m29s">1:42:29</a> เสามงคล😂


---

# 3. Data Preparation + Data Cleansing

In [96]:
df.head(10)

,Name,Comments
0,Woraporn Keawma,ชอบฟังมาก เพลินสุด ชอบตั้งแต่เล่น raknalok
1,Phanuwat Benkasem,พวกคุณมันแค่เด็กในเมือง 555 ไอเหี้ยวันนั้นถ้าอ...
2,Mxx K,เสื้อลุงไนท์หาซื้อจากไหนได้บ้างครับ555
3,หยุดเหมียวไม่ได้,"<a href=""https://www.youtube.com/watch?v=pxFYv..."
4,ฬิน,ขำเกิน ปวดท้อง
5,ูLightvan007,"<a href=""https://www.youtube.com/watch?v=pxFYv..."
6,Razen,บ้านพี่ขยะหน่ะมัน คาด เดา ไม่ ได้~~~~
7,SKT_X,กายนี่คือกายไหนครับเจ้าของบ้าน
8,Sosoมัง,"<a href=""https://www.youtube.com/watch?v=pxFYv..."
9,Fifa Thanakrit,คิดในแง่ดีบ้านเอียงก็ทำให้ได้คอนเทนไปตลอดในวัน...


In [99]:
import os 
save_path = os.path.join(os.getcwd(), 'comments.xlsx')
df.to_excel(save_path, index=False)

https://datarockie.com/blog/sentiment-analysis-bard-ai/